In [12]:
import os
from dotenv import load_dotenv, find_dotenv
import time
import json
import tiktoken
from PyPDF2 import PdfReader
import streamlit as st
from openai import OpenAI
from pydantic import BaseModel, Field
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

In [13]:
token = os.environ["GITHUB_API_TOKEN"]
endpoint = "https://models.github.ai/inference"
client = OpenAI(base_url=endpoint,
    api_key=token)

In [14]:
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text.strip()

In [15]:
def count_tokens(text, encoding_name="o200k_base"):
    enc = tiktoken.get_encoding(encoding_name)
    return len(enc.encode(text))

In [27]:
def llm_call_openai(model_name, messages, max_tokens, output_format):
    response = client.beta.chat.completions.parse(
        messages=messages,
        temperature=0.7,
        max_tokens=max_tokens,
        top_p=0.9,
        model=model_name,
        response_format=output_format
    )
    return response.choices[0].message.content

In [33]:
class structured_output(BaseModel):
    skills: list[str]
    resume_init_suggestions: list[str]

In [ ]:
### STREAMLIT APP - {input, output, and processing}


#st.set_page_config(page_title="Resume Analyzer", page_icon=":mag_right:")
#st.title("Resume Analyzer")
#resume_file = st.file_uploader("Upload your resume (PDF format)", type=["pdf"])

In [34]:
resume_text= extract_text_from_pdf(r"/Users/kathisnehith/Downloads/Shivananda_Dayananda_Resume (2).pdf")
user_prompt= f"Analyze the resume get the suggestions and top skills. \n\n{resume_text}"
system_instruction = """ You are a resume analysis expert. When given a resume document, analyze it to provide:

1. **Suggestive Changes**: 3-4 specific, actionable improvements to enhance the resume's impact. Focus on:
   - Quantifying achievements with metrics/numbers
   - Using Harvard-style action verbs and result-oriented language
   - Improving structure, formatting, or section organization
   - Strengthening weak bullet points with specific accomplishments
   - Adding missing technical keywords relevant to their field

2. **Top 5 Skills**: The most relevant technical and professional skills based on frequency, recency, and industry importance.

For each job in work experience, identify:
- Job title, company, and duration
- Key technologies and skills used
- Quantifiable achievements and impact

Output format: (in given structured format requested)"""

In [35]:

model_name = "openai/gpt-4o-mini"
messages=[
    {"role": "system", "content": system_instruction},
    {"role": "user", "content": user_prompt},
]

skills_extract = llm_call_openai(model_name, messages, 350, structured_output)
print(skills_extract)
s=json.loads(skills_extract)

print(s['skills'])


{"resume_init_suggestions":["Quantify achievements more consistently by including specific metrics for all bullet points, such as the percentage of improvement or cost savings for each project.","Use stronger action verbs at the beginning of each bullet point. For example, replace 'Engineered' with 'Architected' or 'Deployed' with 'Implemented' to convey a stronger sense of leadership and responsibility.","Improve the structure of the Skills section by categorizing skills into sub-sections (e.g., Programming Languages, Cloud Technologies, Databases) for better readability.","Add a 'Technical Keywords' section that highlights industry-relevant terms, such as 'Data Warehousing', 'ETL', 'Machine Learning', or 'Big Data', to enhance visibility in applicant tracking systems."],"skills":["AWS (EC2, S3, Redshift)","Data Engineering (ETL, Data Pipelines)","SQL & NoSQL Databases (PostgreSQL, MongoDB)","Python & Spark Programming","Data Visualization (Grafana, Power BI)"]}
['AWS (EC2, S3, Redshi

In [ ]:
# Store slider values in session_state for later use
for skill in s["skills"]:
    slider_key = f"skill_{skill}"
    st.session_state[slider_key] = st.slider(skill, min_value=0, max_value=10, value=1, key=slider_key)

# Example: Store user input as well
user_notes = st.text_area("Add your notes or context")
st.session_state["user_notes"] = user_notes


In [ ]:
llm_call_openai(model_name, messages, 350, structured_output)